## Prepare flow directions and related data from a DEM

With HydroMT-Wflow, a user can choose to build a model in a geographic or projected coordinate system from an input Digital Elevation Model (DEM) and Flow Direction (flwdir) dataset.

While DEM data are often available, this is not the always the case for the flow directions. In the plugin, we made the choice to build a Wflow model directly from user provided DEM and flwdir datasets rather than reprojecting a DEM and/or deriving flwdir on the fly. This is because there are a lot of available techniques to derive flow directions and we want the user to be sure the flow directions matches the terrain and user expectations.

Because of this, we prefer to provide this notebook as a possible pre-processing step before calling a hydromt build wflow command. We will do this using the different [flow directions methods from HydroMT](https://deltares.github.io/hydromt/latest/api.html#flow-direction-methods) and [PyFlwDir](https://deltares.github.io/pyflwdir/latest/index.html)

### Load dependencies

In [ ]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
# pyflwdir
import pyflwdir
# hydromt
from hydromt import DataCatalog, flw
#plot
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import cartopy.crs as ccrs

In [ ]:
# Common plot settings
plt.style.use("seaborn-whitegrid")  # set nice style
# we assume the model maps are in the geographic CRS EPSG:4326
proj = ccrs.PlateCarree()

# create nice elevation colormap
c_dem = plt.cm.terrain(np.linspace(0.25, 1, 256))
cmap = colors.LinearSegmentedColormap.from_list("dem", c_dem)
norm = colors.Normalize(vmin=0, vmax=2000)
kwargs = dict(cmap=cmap, norm=norm)

# legend settings
legend_kwargs = dict(
    title="Legend",
    loc="lower right",
    frameon=True,
    framealpha=0.7,
    edgecolor="k",
    facecolor="white",
)

def add_legend_titles(ax, title, add_legend=True, projected=True):
    ax.xaxis.set_visible(True)
    ax.yaxis.set_visible(True)
    if projected:
        ax.set_xlabel("Easting [m]")
        ax.set_ylabel("Northing [m]")
    else:
        ax.set_ylabel("latitude [degree north]")
        ax.set_xlabel("longitude [degree east]")
    _ = ax.set_title(title)
    if add_legend:
        legend = ax.legend(**legend_kwargs)

        return legend

### Elevation data and reprojection

In this example we will use the `merit_hydro_1k` data in the pre-defined `artifact_data` catalog of HydroMT. 

If you prefer to have a Wflow model in projected coordinate system (CRS in metres), you will need first to reproject the DEM and we will see here an example of how to do this.

First let's read the data:

In [ ]:
data_catalog = DataCatalog("artifact_data")
merit = data_catalog.get_rasterdataset("merit_hydro_1k", bbox = [11.8,46.0,12.3,46.2])

In [ ]:
# plot
# initialize image with geoaxes
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(projection=proj)

## plot elevation\
merit['elevtn'].plot(
    transform=proj, ax=ax, zorder=1, cbar_kwargs=dict(aspect=30, shrink=0.8), **kwargs
)

# plot flwdir
flwdir = flw.flwdir_from_da(merit["flwdir"], ftype="infer", check_ftype=True)
feat = flwdir.streams()
gdf = gpd.GeoDataFrame.from_features(feat, crs=merit.raster.crs)
gdf.plot(ax=ax, color="blue", linewidth=0.5, zorder=2, label="Flow directions")

legend = add_legend_titles(ax, "MERIT Hydro IHU 1km", projected=False)

And now let's reproject the DEM to the projected CRS EPSG 3857 using [hydromt.DataArray.raster.reproject](https://deltares.github.io/hydromt/latest/_generated/hydromt.DataArray.raster.reproject.html#hydromt.DataArray.raster.reproject) method.

In [ ]:
merit_elv_utm = merit["elevtn"].raster.reproject(
    dst_crs=3857,
    dst_res = None, # keep the same resolution
    method = 'bilinear',
)

### Reprojecting flow directions

If your original data did contain flow directions, you can decide to reproject rather than re-create them from the reprojected DEM.

To do this, you can use the [hydromt.flw.reproject_hydrography_like](https://deltares.github.io/hydromt/latest/_generated/hydromt.flw.reproject_hydrography_like.html#hydromt.flw.reproject_hydrography_like) method from HydroMT.

This method aims to conserve flow direction during reprojection by deriving flwdir from a reprojected grid of synthetic elevation, based on the log10  of upstream area [m2].

In [ ]:
merit_utm_reproj = flw.reproject_hydrography_like(
    ds_hydro = merit[["flwdir", "uparea"]],
    da_elv = merit_elv_utm,
)

Let's compare our new reprojected flow directions to the original ones

In [ ]:
# plot
# initialize image with geoaxes
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot()

## plot elevation\
merit_elv_utm.plot(
    ax=ax, zorder=1, cbar_kwargs=dict(aspect=30, shrink=0.8), **kwargs
)

# plot flwdir
flwdir = flw.flwdir_from_da(merit["flwdir"], ftype="infer", check_ftype=True)
feat = flwdir.streams(min_sto=2)
gdf = gpd.GeoDataFrame.from_features(feat, crs=merit.raster.crs)
gdf.to_crs(merit_elv_utm.raster.crs).plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Blues(np.linspace(0.4, 1, 7))), label="Flow directions")

# plot flwdir reproj
flwdirr = flw.flwdir_from_da(merit_utm_reproj["flwdir"], ftype="infer", check_ftype=True)
featr = flwdirr.streams(min_sto=2)
gdfr = gpd.GeoDataFrame.from_features(featr, crs=merit_utm_reproj.raster.crs)
gdfr.plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Reds(np.linspace(0.4, 1, 7))), label="Flow directions reprojected")

legend = add_legend_titles(ax, "MERIT Hydro IHU 1km Reprojected")

### Deriving flow directions from DEM

To derive flow directions from a DEM, you can use the [hydromt.flw.d8_from_dem](https://deltares.github.io/hydromt/latest/_generated/hydromt.flw.d8_from_dem.html#hydromt.flw.d8_from_dem) method of HydroMT.

This method derives D8 flow directions grid from an elevation grid and allows several options to the users:
 - **outlets**: outlets can be defined at ``edge``s of the grid or force all flow to go to the minimum elevation point ``min``. Additionnally, the user can also specify the pits locations via ``idxs_pit``.
 - **river burning**: it is possible to provide a river vector layer ``gdf_stream`` with ``uparea`` (km2) column which is used to burn the river in the elevation data.
 - **depression filling**: local depressions are filled based on their lowest pour point level if the pour point depth is smaller than the maximum pour point depth ``max_depth``, otherwise the lowest elevation in the depression becomes a pit.

Let's see an example:

In [ ]:
# Start a merit_utm dataset
merit_utm_flw = merit_elv_utm.to_dataset(name="elevtn")
# Derive flow directions with outlets at the edges
merit_utm_flw["flwdir_edge"] = flw.d8_from_dem(
    da_elv = merit_elv_utm,
    gdf_stream = None,
    max_depth=-1,
    outlets = "edge",
    idxs_pit = None,
)
# Derive flow directions with outlet at the min elevation edge cell
merit_utm_flw["flwdir_min"] = flw.d8_from_dem(
    da_elv = merit_elv_utm,
    gdf_stream = None,
    max_depth=-1,
    outlets = "min",
    idxs_pit = None,
)

Let's plot all the different methods

In [ ]:
# plot
# initialize image with geoaxes
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot()

## plot elevation\
merit_elv_utm.plot(
    ax=ax, zorder=1, cbar_kwargs=dict(aspect=30, shrink=0.8), alpha=0.5, **kwargs
)

# plot flwdir
flwdir = flw.flwdir_from_da(merit["flwdir"], ftype="infer", check_ftype=True)
feat = flwdir.streams(min_sto=2)
gdf = gpd.GeoDataFrame.from_features(feat, crs=merit.raster.crs)
gdf.to_crs(merit_elv_utm.raster.crs).plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Blues(np.linspace(0.4, 1, 7))), label="Flow directions")

# plot flwdir reproj
flwdirr = flw.flwdir_from_da(merit_utm_reproj["flwdir"], ftype="infer", check_ftype=True)
featr = flwdirr.streams(min_sto=2)
gdfr = gpd.GeoDataFrame.from_features(featr, crs=merit_utm_reproj.raster.crs)
gdfr.plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Reds(np.linspace(0.4, 1, 7))), label="Flow directions reprojected")

# plot flwdir edge
flwdir_edge = flw.flwdir_from_da(merit_utm_flw["flwdir_edge"], ftype="infer", check_ftype=True)
feate = flwdir_edge.streams(min_sto=2)
gdfe = gpd.GeoDataFrame.from_features(feate, crs=merit_utm_flw.raster.crs)
gdfe.plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Greens(np.linspace(0.4, 1, 7))), label="Flow directions edge")

# plot flwdir min
flwdir_min = flw.flwdir_from_da(merit_utm_flw["flwdir_min"], ftype="infer", check_ftype=True)
featm = flwdir_min.streams(min_sto=2)
gdfm = gpd.GeoDataFrame.from_features(featm, crs=merit_utm_flw.raster.crs)
gdfm.plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Purples(np.linspace(0.4, 1, 7))), label="Flow directions min")

legend = add_legend_titles(ax, "MERIT Hydro IHU 1km Reprojected")

Now let's see an example where we want to burn rivers.
In the data artifact, we have one river vector database : ``rivers_lin2019_v1``.

Let's use it for burning.

In [ ]:
# Read the data from data catalog
rivers = data_catalog.get_geodataframe("rivers_lin2019_v1", bbox=[11.8,46.0,12.3,46.2])

# In this dataset, there is an Area column representing upstream area in km2
# We need to rename it to uparea
rivers = rivers.rename(columns={"Area": "uparea"})
# And finally make sure rivers is reprojected to the same CRS as the elevation
rivers = rivers.to_crs(merit_elv_utm.raster.crs)

# Now let's use it to derive flow directions
merit_utm_flw["flwdir_riverburn"] = flw.d8_from_dem(
    da_elv = merit_elv_utm,
    gdf_stream = rivers,
    max_depth=-1,
    outlets = "edge",
    idxs_pit = None,
)

In [ ]:
# plot
# initialize image with geoaxes
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot()

## plot elevation\
merit_elv_utm.plot(
    ax=ax, zorder=1, cbar_kwargs=dict(aspect=30, shrink=0.8), alpha=0.5, **kwargs
)

# plot flwdir
flwdir = flw.flwdir_from_da(merit["flwdir"], ftype="infer", check_ftype=True)
feat = flwdir.streams(min_sto=2)
gdf = gpd.GeoDataFrame.from_features(feat, crs=merit.raster.crs)
gdf.to_crs(merit_elv_utm.raster.crs).plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Blues(np.linspace(0.4, 1, 7))), label="Flow directions")

# plot flwdir reproj
flwdirr = flw.flwdir_from_da(merit_utm_reproj["flwdir"], ftype="infer", check_ftype=True)
featr = flwdirr.streams(min_sto=2)
gdfr = gpd.GeoDataFrame.from_features(featr, crs=merit_utm_reproj.raster.crs)
gdfr.plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Reds(np.linspace(0.4, 1, 7))), label="Flow directions reprojected")

# plot flwdir edge
flwdir_edge = flw.flwdir_from_da(merit_utm_flw["flwdir_edge"], ftype="infer", check_ftype=True)
feate = flwdir_edge.streams(min_sto=2)
gdfe = gpd.GeoDataFrame.from_features(feate, crs=merit_utm_flw.raster.crs)
gdfe.plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Greens(np.linspace(0.4, 1, 7))), label="Flow directions edge")

# plot flwdir riverburn
flwdir_min = flw.flwdir_from_da(merit_utm_flw["flwdir_riverburn"], ftype="infer", check_ftype=True)
featm = flwdir_min.streams(min_sto=2)
gdfm = gpd.GeoDataFrame.from_features(featm, crs=merit_utm_flw.raster.crs)
gdfm.plot(ax=ax, column="strord", cmap=colors.ListedColormap(cm.Purples(np.linspace(0.4, 1, 7))), label="Flow directions river burning")

rivers.plot(ax=ax, color="black", label="Rivers Lin")

legend = add_legend_titles(ax, "MERIT Hydro IHU 1km Reprojected")

## Deriving other DEM and flow directions related data

Once you are satisfied with your flow direction map, you can create additionnal derived variables like upstream area or streamorder that can prove useful for example to build a model based on ``subbasin`` region.

Here are some examples how to do that using PyFlwdir methods.

In [ ]:
# Create a dataset with the riverburn flow directions
merit_utm = merit_elv_utm.to_dataset(name="elevtn")
merit_utm["flwdir"] = merit_utm_flw["flwdir_riverburn"]
dims = merit_utm.raster.dims

# Create a PyFlwDir object from the dataset
flwdir = flw.flwdir_from_da(merit_utm["flwdir"])

# uparea
uparea = flwdir.upstream_area(unit="km2")
merit_utm["uparea"] = xr.Variable(dims, uparea, attrs=dict(_FillValue=-9999))

# stream order
strord = flwdir.stream_order()
merit_utm["strord"] = xr.Variable(dims, strord)
merit_utm["strord"].raster.set_nodata(255)

# slope
slope = pyflwdir.dem.slope(
    elevtn=merit_utm["elevtn"].values,
    nodata = merit_utm["elevtn"].raster.nodata,
    latlon = False, # True if geographic crs, False if projected crs
    transform = merit_utm["elevtn"].raster.transform,
)
merit_utm["slope"] = xr.Variable(dims, slope)
merit_utm["slope"].raster.set_nodata(merit_utm["elevtn"].raster.nodata)

# basin at the pits locations
basins = flwdir.basins(idxs=flwdir.idxs_pit).astype(np.int32)
merit_utm["basins"] = xr.Variable(dims, basins, attrs=dict(_FillValue=0))

merit_utm

In [ ]:
# plot
fig, axes = plt.subplots(3, 2, figsize=(15, 15))

## plot elevation
merit_utm["elevtn"].plot(
    ax=axes[0,0], zorder=1, cbar_kwargs=dict(aspect=30, shrink=0.8), alpha=0.5, **kwargs
)
_ = add_legend_titles(axes[0,0], "Elevation [m asl]", add_legend=False)

# plot flwdir riverburn
flwdir = flw.flwdir_from_da(merit_utm["flwdir"], ftype="infer", check_ftype=True)
feat = flwdir.streams(min_sto=2)
gdf = gpd.GeoDataFrame.from_features(feat, crs=merit_utm.raster.crs)
gdf.to_crs(merit_elv_utm.raster.crs).plot(ax=axes[0,1], column="strord", cmap=colors.ListedColormap(cm.Blues(np.linspace(0.4, 1, 7))), label="Flow directions")
_ = add_legend_titles(axes[0,1], "Flow directions", add_legend=False)

# plot uparea
merit_utm["uparea"].plot(ax=axes[1,0])
_ = add_legend_titles(axes[1,0], "Upstream area [km2]", add_legend=False)

# plot strord
merit_utm["strord"].plot(ax=axes[1,1], cmap=colors.ListedColormap(cm.Blues(np.linspace(0.4, 1, 7))))
_ = add_legend_titles(axes[1,1], "Strahler Stream order", add_legend=False)

# plot slope
merit_utm["slope"].plot(ax=axes[2,0])
_ = add_legend_titles(axes[2,0], "Slope [m/m]", add_legend=False)

# plot basins
merit_utm["basins"].plot(ax=axes[2,1])
_ = add_legend_titles(axes[2,1], "Basins ID", add_legend=False)
